<a href="https://colab.research.google.com/github/JacobChunn/trading-transformer/blob/master/trading-transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trading Transformer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import pandas as pd

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [ ]:
class NextValuePredictor(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(NextValuePredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.fc = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, tgt):
        #print("tgt", tgt)
        seq_length = tgt.size(1)
        #print("seq_length", seq_length)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(2) & nopeak_mask
        #print("tgt_mask: ", tgt_mask)
        #print("tgt_mask.shape: ", tgt_mask.shape)
        return tgt_mask

    def forward(self, tgt):
        tgt_mask = self.generate_mask(tgt)
        tgt_embedded = self.dropout(self.positional_encoding(self.embedding(tgt)))
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, tgt_mask)
        output = self.fc(dec_output)
        return output

In [ ]:
# Read and preprocess the data
data = pd.read_csv('EURUSD-short.txt', skiprows=1, header=None)
data.columns = ['<TICKER>', '<DTYYYYMMDD>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<VOL>']
data.columns = [col.strip('<>') for col in data.columns]

data['TIME'] = data['TIME'].apply(lambda x: f'{int(x):06d}')
data['DATETIME'] = pd.to_datetime(data['DTYYYYMMDD'].astype(str) + data['TIME'], format='%Y%m%d%H%M%S')

# Extract relevant columns
data = data[['DATETIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']]

In [ ]:
# Create sequences using sliding window
def create_sequences(data, window_size):
    sequences = []
    for i in range(len(data) - window_size):
        sequences.append(data.iloc[i:i + window_size].values)
    return np.array(sequences)

# Select a batch of sequences for the tensor
def get_batch(sequences, batch_size):
    indices = np.random.choice(len(sequences), batch_size, replace=False)
    return sequences[indices]

window_size = 50  # Number of 60-second intervals in a day
batch_size = 64    # Number of sequences per batch
vocab_size = 20000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 512  # orig val is 2048
max_seq_length = 390  # Set to window_size - need to look into how this relates to window_size
dropout = 0.1

sequences = create_sequences(data["OPEN"], window_size)
batch_sequences = get_batch(sequences, batch_size)

# Convert to PyTorch tensor
data_tensor = torch.tensor(batch_sequences, dtype=torch.float32)

data_tensor_int = (data_tensor * 10000).long()

In [ ]:
# Model instance
model = NextValuePredictor(vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Training
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss() #using cross entropy instead
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
train_input = data_tensor_int[:, :-1]
train_target = data_tensor_int[:, 1:]

In [ ]:
model.train()

for epoch in range(25):
    optimizer.zero_grad()
    output = model(train_input)
    loss = criterion(output.contiguous().view(-1, vocab_size), train_target.contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 10.055262565612793
Epoch: 2, Loss: 9.629905700683594
Epoch: 3, Loss: 9.2384614944458
Epoch: 4, Loss: 8.89424991607666
Epoch: 5, Loss: 8.601384162902832
Epoch: 6, Loss: 8.328292846679688
Epoch: 7, Loss: 8.088665008544922
Epoch: 8, Loss: 7.856785297393799
Epoch: 9, Loss: 7.634387493133545
Epoch: 10, Loss: 7.404182434082031
Epoch: 11, Loss: 7.194997310638428
Epoch: 12, Loss: 6.980340957641602
Epoch: 13, Loss: 6.782764434814453
Epoch: 14, Loss: 6.586864948272705
Epoch: 15, Loss: 6.3984479904174805
Epoch: 16, Loss: 6.220820426940918
Epoch: 17, Loss: 6.04638671875
Epoch: 18, Loss: 5.881972789764404
Epoch: 19, Loss: 5.721706867218018
Epoch: 20, Loss: 5.568544387817383
Epoch: 21, Loss: 5.409043312072754
Epoch: 22, Loss: 5.26797342300415
Epoch: 23, Loss: 5.130000114440918
Epoch: 24, Loss: 4.993512153625488
Epoch: 25, Loss: 4.860896587371826


In [ ]:
val_sequences = get_batch(sequences, batch_size)
val_data = torch.tensor(val_sequences, dtype=torch.float32)
val_data_int = (val_data * 10000).long()

val_input = val_data_int[:, :-1]
val_target = val_data_int[:, 1:]

In [ ]:
model.eval()

with torch.no_grad():
    val_output = model(val_input)
    val_loss = criterion(val_output.contiguous().view(-1, vocab_size), val_target.contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")
    print("input:\n")
    print(val_data_int)
    print(val_data_int.shape)
    print("output:\n")
    print(val_output)
    print(val_output.shape)

Validation Loss: 6.693717002868652
input:

tensor([[10860, 10860, 10860,  ..., 10861, 10860, 10860],
        [10933, 10932, 10933,  ..., 10933, 10933, 10933],
        [10702, 10700, 10700,  ..., 10700, 10702, 10699],
        ...,
        [10775, 10775, 10775,  ..., 10778, 10778, 10778],
        [10932, 10932, 10932,  ..., 10933, 10934, 10934],
        [10935, 10935, 10935,  ..., 10935, 10935, 10935]])
torch.Size([64, 50])
output:

tensor([[[-8.3823e-02, -3.3413e-02, -2.9529e-01,  ..., -5.0217e-01,
          -7.6658e-01,  1.3690e-01],
         [-1.4010e-02, -9.6941e-04, -3.4428e-01,  ..., -4.9619e-01,
          -6.3659e-01,  1.7658e-01],
         [-4.8548e-02,  2.6456e-02, -4.5473e-01,  ..., -5.1016e-01,
          -5.3592e-01,  1.9983e-01],
         ...,
         [-5.5579e-01, -1.7557e+00, -1.8257e+00,  ..., -6.5025e-02,
          -1.0667e+00, -6.7644e-01],
         [-6.0225e-01, -1.8032e+00, -1.8333e+00,  ..., -2.9712e-02,
          -1.1249e+00, -6.3731e-01],
         [-6.3386e-01, -5.